In [1]:
import numpy as np
import rasterio
from PIL import Image
from rasterio.plot import show
import matplotlib.pyplot as plt


#Open TIF image

img = rasterio.open("Coregistration2015-2018.tif")
imgarray = img.read(1)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:321: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [2]:
img.read().shape

(2, 16455, 25801)

In [3]:
imgarray = img.read(1)

imgarray = 10*np.log10(imgarray)  # données au format logarithme

l, w = imgarray.shape[0], imgarray.shape[1]

# on fait la même chose pour toutes les images qu'il reste

empty_img = np.zeros((l, w))  # l'image à colorer à chaque changement


# couleur est un tableau de 4 éléments RGB : ex [66, 83, 157, 255]


C:\Users\User\AppData\Local\Temp\ipykernel_13408\692425588.py:3: RuntimeWarning: divide by zero encountered in log10
  imgarray = 10*np.log10(imgarray)  # données au format logarithme


In [4]:
empty_img = np.zeros((l, w, 3))  # l'image à colorer à chaque changement

# 2dB en moins par rapport à la valeur initiale
arrayseuil = imgarray-4


In [5]:
len(empty_img[0][0])

3

In [6]:
img_array_late1_wdb = img.read(2)
img_array_late1 = 10*np.log10(img_array_late1_wdb)

C:\Users\User\AppData\Local\Temp\ipykernel_13408\3653115308.py:2: RuntimeWarning: divide by zero encountered in log10
  img_array_late1 = 10*np.log10(img_array_late1_wdb)
C:\Users\User\AppData\Local\Temp\ipykernel_13408\3653115308.py:2: RuntimeWarning: invalid value encountered in log10
  img_array_late1 = 10*np.log10(img_array_late1_wdb)


In [7]:
img_array_late1_wdb = (img_array_late1_wdb + 0.37)*255

In [8]:
img_array_late1_wdb[img_array_late1_wdb > 255] = 255
img_array_late1_wdb[img_array_late1_wdb < 0] = 0

In [9]:
tif_to_rgb = np.zeros((l, w, 3))

tif_to_rgb[:, :, 0] = img_array_late1_wdb
tif_to_rgb[:, :, 1] = img_array_late1_wdb
tif_to_rgb[:, :, 2] = img_array_late1_wdb

img_array_late1_wdb = tif_to_rgb

In [10]:
#empty_img = img_array_late1_wdb.copy()
#empty_img2 = img_array_late1_wdb.copy()

In [11]:
print(np.count_nonzero(empty_img))

0


In [12]:
img_array_late1_wdb.shape

(16455, 25801, 3)

In [13]:

def compare(imgarray_late, arrayseuil, couleur, empty_img):
    comp = (imgarray_late < arrayseuil)
     
    empty_img[comp == True] = couleur
    
    arrayseuil[comp == True] = -np.inf
        
    return empty_img, arrayseuil


def last_compare(imgarray_last, arrayseuil, couleur, imgarray_png, img_array_late1_wdb):
    comp = (imgarray_last < arrayseuil)
     
    empty_img[comp == True] = couleur
    
    #empty_img[comp == False] = img_array_late1_wdb[comp == False]
    return empty_img

# on finit de colorer empty_img avec les valeurs que l'on trouve dans l'image de départ
# faire attention au format des données que l'on obtient avec rasterio


def fill_base_image():
    for i in range(l):
        for j in range(w):
            if empty_img[i][j] == np.inf:
                empty_img[i][j] = imgarray[i][j]


img_array_late1_wdb = img.read(2) #2018


img2 = rasterio.open("Coregistration2015-2016.tif")
img_array_late2 = img2.read(2)

img_array_late2 = 10*np.log10(img_array_late2) #2016

img_png = rasterio.open("2015RGB.tif")
imgarray_png=img_png.read(1)

empty_img = tif_to_rgb

empty_img = compare( img_array_late1, arrayseuil, (255, 0, 0), empty_img)[0]

empty_img = last_compare( img_array_late2, arrayseuil, (0, 255, 0), empty_img, img_array_late1_wdb)


C:\Users\User\AppData\Local\Temp\ipykernel_13408\2954943634.py:36: RuntimeWarning: divide by zero encountered in log10
  img_array_late2 = 10*np.log10(img_array_late2) #2016
C:\Users\User\AppData\Local\Temp\ipykernel_13408\2954943634.py:36: RuntimeWarning: invalid value encountered in log10
  img_array_late2 = 10*np.log10(img_array_late2) #2016


In [14]:
empty_img.shape

int_image = empty_img.astype(np.uint8)


In [17]:
np.mean(img_array_late1_wdb)
np.max(img_array_late1_wdb)

92.89551

In [18]:
np.mean(img_array_late1_wdb)

0.13462096

In [19]:
np.mean(img_array_late1_wdb)

0.13462096

In [20]:
np.max(tif_to_rgb)

255.0

In [25]:
pil_image = Image.fromarray(empty_img.astype(np.uint8))
pil_image.show()

In [22]:
img_png = rasterio.open("2015RGB.tif")
imgarray_png=img_png.read(1)
print(imgarray_png.shape)

(908, 1313)
